# LLM as Player

In [1]:
%pip install openai ollama -q

/Users/baloghbence/Documents/Projects/maverick/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import logging, sys, os

from openai import OpenAI
from openai.types.responses.response_usage import ResponseUsage
from dotenv import load_dotenv

from maverick import (
    Game,
    Player,
    ActionType,
    GameState,
    PlayerLike,
    PlayerAction,
    PlayerState,
)
from maverick.players import FoldBot, CallBot, AggressiveBot
from maverick.utils import estimate_holding_strength, find_highest_scoring_hand

_ = load_dotenv(override=True)

In [7]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"
#model_name = "llama3.2"
#model_name = "gpt-oss:latest"
model_name = "deepseek-r1:8b"
llm = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

# model_name = "gpt-5-nano"
# api_key = os.getenv('OPENAI_API_KEY')
# llm = OpenAI(api_key=api_key)

In [8]:
response = llm.responses.parse(
    model=model_name, 
    input=[{"role": "user", "content": "Are you ready to play poker?"}]
)
response.output_text, response.usage.total_tokens

("Hey, I'm ready to play poker anytime! But since we're interacting via text, I'll need to handle everything with descriptions, card assignments, and betting. We can play a Texas Hold'em game if you're up for that, or you can suggest another variation.\n\nTo get started, let's:\n\n- **Choose a game type:** Texas Hold'em is a good starting point.\n- **Decide on the number of players:** You can be one player, and I'll manage the rest of the actions.\n- **Set the rules if needed:** For simplicity, we'll use a standard 52-card deck, blinds, betting rounds, etc.\n\nReady when you are—just let me know how you want to proceed, or I can kick off a quick round! 😊",
 542)

In [12]:
from collections import defaultdict


def openai_call_cost(
    *,
    input_tokens: int,
    output_tokens: int,
    model: str,
    cached_input_tokens: int = 0,
) -> float:
    PRICES = defaultdict(
        lambda: {"input": 0.0, "cached": 0.0, "output": 0.0},
        {
            "gpt-5.2":    {"input": 1.75, "cached": 0.175, "output": 14.00},
            "gpt-5.1":    {"input": 1.25, "cached": 0.125, "output": 10.00},
            "gpt-5":      {"input": 1.25, "cached": 0.125, "output": 10.00},
            "gpt-5-mini": {"input": 0.25, "cached": 0.025, "output": 2.00},
            "gpt-5-nano": {"input": 0.05, "cached": 0.005, "output": 0.40},
        },
    )

    p = PRICES[model]

    uncached_input = max(0, input_tokens - cached_input_tokens)

    total_cost =  (
        uncached_input / 1_000_000 * p["input"]
        + cached_input_tokens / 1_000_000 * p["cached"]
        + output_tokens / 1_000_000 * p["output"]
    )
    
    return total_cost
    

openai_call_cost(
    input_tokens=response.usage.input_tokens,
    output_tokens=response.usage.output_tokens,
    model=model_name,
    cached_input_tokens=response.usage.input_tokens_details.cached_tokens if response.usage.input_tokens_details else 0,
)

0.0

In [13]:
SYSTEM_MESSAGE = """
You are an expert Texas Hold'em poker agent.

You will be given the current game state and a list of valid actions.
Your task: choose exactly one valid action and (if required) a numeric amount.

Action semantics:
- FOLD: provide no `amount`.
- CHECK: provide no `amount`.
- CALL: provide no `amount` (the engine infers call cost).
- BET: provide `amount` = the bet size you add now (chips placed this action).
- RAISE: provide `amount` = the raise size BY (increment over the current bet), not "raise to".
  The amount MUST be >= min_raise_by.
  
Card notation:
- Cards use standard short codes: Ah=Ace of hearts, Td=Ten of diamonds, 2c=Two of clubs.
- Ranks: 2-9,T,J,Q,K,A. Suits: h,d,c,s.
  
Glossary of terms:
- equity_vs_field = probability that your hand wins against all remaining players,
  estimated via Monte-Carlo simulation.

Output requirements:
- Output MUST conform to the PlayerAction schema.
- Do NOT output explanations or any extra text.
- Never choose an action not present in valid_actions.
"""


class LLM(Player):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._reset_usage()
        
    def _reset_usage(self) -> None:
        """Reset the usage statistics for this player."""
        self._usage = {
            "input_tokens": 0,
            "output_tokens": 0,
            "total_tokens": 0,
            "input_tokens_details": {
                "cached_tokens": 0,
            },
            "output_tokens_details": {
                "reasoning_tokens": 0
            }
        }
    
    def _increment_usage(self, usage: ResponseUsage) -> None:
        self._usage["input_tokens"] += usage.input_tokens
        self._usage["output_tokens"] += usage.output_tokens
        self._usage["total_tokens"] += usage.total_tokens
        if usage.input_tokens_details:
            self._usage["input_tokens_details"]["cached_tokens"] += usage.input_tokens_details.cached_tokens
        if usage.output_tokens_details:
            self._usage["output_tokens_details"]["reasoning_tokens"] += usage.output_tokens_details.reasoning_tokens
        
    def _total_call_cost(self) -> float:
        """Calculate the total cost of all OpenAI calls made by this player."""
        input_tokens = self._usage["input_tokens"]
        output_tokens = self._usage["output_tokens"]
        cached_input_tokens = self._usage["input_tokens_details"]["cached_tokens"]
        
        return openai_call_cost(
            input_tokens=input_tokens,
            output_tokens=output_tokens,
            model=model_name,
            cached_input_tokens=cached_input_tokens,
        )
    
    def decide_action(
        self,
        game_state: "GameState",
        valid_actions: list["ActionType"],
        min_raise: int,
    ) -> "PlayerAction":
        # ---------- Derived decision-critical numbers ----------
        amount_to_call = max(0, int(game.state.current_bet) - int(self.state.current_bet))
        min_raise_by = int(min_raise)

        # ---------- Debug prints (optional) ----------
        print(f"\n{self.name} is requested to take action.")
        print("     Current street:", game.state.street.name)
        print("     Button position:", game.state.button_position)
        print("     Pot:", game.state.pot)
        print("     Current bet:", game.state.current_bet)
        print("     Small blind:", game.state.small_blind)
        print("     Big blind:", game.state.big_blind)
        print("     Community cards:", [str(c) for c in game.state.community_cards])
        print("     Player stack:", self.state.stack)
        print("     Player current bet:", self.state.current_bet)
        print("     Amount to call:", amount_to_call)
        print("     Private cards:", [str(c) for c in self.state.holding.cards])
        print("     Valid actions:", [v.name for v in valid_actions])
        print("     Min raise BY:", min_raise_by)

        private_cards = self.state.holding.cards
        community_cards = game.state.community_cards

        # ---------- Hand evaluation (your existing utilities) ----------
        strongest_hand, strongest_hand_type, strongest_hand_score = find_highest_scoring_hand(
            private_cards, community_cards, n_min_private=0
        )

        hand_prob = estimate_holding_strength(
            private_cards,
            community_cards=community_cards,
            n_min_private=0,
            n_simulations=1000,
            n_players=len(game.state.get_players_in_hand()),
        )

        print("     Best hand type:", strongest_hand_type.name)
        print("     Best hand score:", strongest_hand_score)
        print("     Equity (MC):", hand_prob)

        # ---------- Prompt (structured, minimal, unambiguous) ----------
        player_message = f"""
STATE (chip amounts are integers):

player:
  id: {self.id}
  name: {self.name}
  stack: {int(self.state.stack)}
  current_bet: {int(self.state.current_bet)}
  amount_to_call: {amount_to_call}
  hole_cards: {', '.join(card.code() for card in private_cards)}

table:
  street: {game.state.street.name}
  community_cards: {', '.join(card.code() for card in community_cards)}
  pot: {int(game.state.pot)}
  current_bet: {int(game.state.current_bet)}
  blinds: [{int(game.state.small_blind)}, {int(game.state.big_blind)}]
  players_in_hand: {len(game.state.get_players_in_hand())}

constraints:
  valid_actions: {[v.name for v in valid_actions]}
  min_raise_by: {min_raise_by}

hand_eval:
  best_current_hand: {strongest_hand}
  best_hand_type: {strongest_hand_type.name}
  equity_vs_field: {hand_prob:.3f}

Choose your next action now.
If action is BET or RAISE, include integer amount.
Remember: for RAISE, amount means "raise BY" and must be >= min_raise_by.
"""

        # ---------- Call the model ----------
        try:
            response = llm.responses.parse(
                model=model_name,
                input=[
                    {"role": "system", "content": SYSTEM_MESSAGE},
                    {"role": "user", "content": player_message},
                ],
                text_format=PlayerAction,
            )
            self._increment_usage(response.usage)
            action: PlayerAction = response.output_parsed
            print(f"     LLMPlayer raw action output: {action}")  
        except Exception as e:
            logging.error(f"LLMPlayer {self.id} failed to decide action: {e}")
            action = PlayerAction(player_id=self.id, action_type=ActionType.FOLD)

        # ---------- Post-parse validation & self-heal ----------
        valid_set = set(valid_actions)
        if action.action_type not in valid_set:
            # safest fallback: CHECK if possible else FOLD
            logging.error("LLMPlayer produced invalid action type, applying fallback.")
            fallback = ActionType.CHECK if ActionType.CHECK in valid_set else ActionType.FOLD
            action = PlayerAction(player_id=self.id, action_type=fallback)

        needs_amount = action.action_type in {ActionType.BET, ActionType.RAISE}
        if needs_amount:
            # Missing amount -> pick a conservative default
            if getattr(action, "amount", None) is None:
                logging.error("LLMPlayer missing amount for bet/raise, applying default.")
                if action.action_type == ActionType.RAISE:
                    action.amount = min_raise_by
                else:
                    # BET default: small probe (e.g., 1/3 pot) but bounded by stack
                    suggested = max(1, int(game.state.pot) // 3)
                    action.amount = min(suggested, int(self.state.stack))
            # Normalize to int and clamp
            action.amount = int(action.amount)

            # Enforce min raise-by
            if action.action_type == ActionType.RAISE and action.amount < min_raise_by:
                action.amount = min_raise_by

            # No negative or zero bet/raise
            if action.amount <= 0:
                logging.error("LLMPlayer produced non-positive amount for bet/raise, applying fallback.")
                fallback = ActionType.CHECK if ActionType.CHECK in valid_set else ActionType.FOLD
                action = PlayerAction(player_id=self.id, action_type=fallback)

            # Can't wager more than stack in this action (if your engine allows all-in as exact stack, keep this)
            if action.amount > int(self.state.stack):
                logging.error("LLMPlayer produced amount exceeding stack, clamping to stack.")
                action.amount = int(self.state.stack)

        else:
            # Strip stray amount for non-sizing actions
            if hasattr(action, "amount") and action.amount is not None:
                logging.error("LLMPlayer produced amount for non-sizing action, stripping.")
                action.amount = None

        print(f"     Player decides to {action.action_type.name}"
              f"{'' if getattr(action, 'amount', None) is None else f' ({action.amount})'}.\n")
        sys.stdout.flush()
        return action


In [14]:
# Configure logging such that we only get the log messages of the game
logging.basicConfig(
    level=logging.DEBUG,
    format="[%(levelname)s] %(name)s: %(message)s",
)
logging.getLogger().setLevel(logging.WARNING)  # Set root logger to WARNING
logging.getLogger("maverick").setLevel(logging.DEBUG)  # Only maverick logs at DEBUG

# Create game with blinds
game = Game(small_blind=10, big_blind=20, max_hands=40)

# Create and add players with different strategies
players: list[PlayerLike] = [
    CallBot(id="p1", name="CallBot", state=PlayerState(stack=1000, seat=0)),
    AggressiveBot(id="p2", name="AggroBot", state=PlayerState(stack=1000, seat=1)),
    LLM(id="p3", name=model_name, state=PlayerState(stack=1000, seat=2)),
    FoldBot(id="p4", name="FoldBot", state=PlayerState(stack=1000, seat=3)),
]

for player in players:
    game.add_player(player)
    
game.start()

[INFO] maverick: Player CallBot joined the game.
[INFO] maverick: Player AggroBot joined the game.
[INFO] maverick: Player deepseek-r1:8b joined the game.
[INFO] maverick: Player FoldBot joined the game.
[INFO] maverick: Game started.

[INFO] maverick: ============================== Hand 1 ==============================

[INFO] maverick: PRE_FLOP | Dealing hole cards. Button: CallBot
[INFO] maverick: PRE_FLOP | Posting small blind of 10 by player AggroBot. Remaining stack: 990
[INFO] maverick: PRE_FLOP | Posting big blind of 20 by player deepseek-r1:8b. Remaining stack: 980
[INFO] maverick: PRE_FLOP | Player FoldBot folds.
[INFO] maverick: PRE_FLOP | Current pot: 30 | Current bet: 20
[INFO] maverick: PRE_FLOP | Player CallBot calls with amount 20. Remaining stack: 980.
[INFO] maverick: PRE_FLOP | Current pot: 50 | Current bet: 20
[INFO] maverick: PRE_FLOP | Player AggroBot raises to amount 50. Remaining stack: 950.
[INFO] maverick: PRE_FLOP | Current pot: 90 | Current bet: 50



deepseek-r1:8b is requested to take action.
     Current street: PRE_FLOP
     Button position: 0
     Pot: 90
     Current bet: 50
     Small blind: 10
     Big blind: 20
     Community cards: []
     Player stack: 980
     Player current bet: 20
     Amount to call: 30
     Private cards: ['J♠', 'A♠']
     Valid actions: ['FOLD', 'CALL', 'RAISE', 'ALL_IN']
     Min raise BY: 70
     Best hand type: HIGH_CARD
     Best hand score: 100.1411
     Equity (MC): 0.51


[ERROR] root: LLMPlayer produced invalid action type, applying fallback.


     LLMPlayer raw action output: player_id='p3' action_type=<ActionType.CHECK: 2> amount=70
     Player decides to FOLD.



[INFO] maverick: PRE_FLOP | Player deepseek-r1:8b folds.
[INFO] maverick: PRE_FLOP | Current pot: 90 | Current bet: 50
[INFO] maverick: PRE_FLOP | Player CallBot calls with amount 30. Remaining stack: 950.
[INFO] maverick: PRE_FLOP | Current pot: 120 | Current bet: 50
[INFO] maverick: PRE_FLOP | Betting round complete

[INFO] maverick: FLOP | Dealt flop. Community cards: ['3♦', '10♠', '8♣']
[INFO] maverick: FLOP | Player AggroBot bets amount 40. Remaining stack: 910.
[INFO] maverick: FLOP | Current pot: 160 | Current bet: 40
[INFO] maverick: FLOP | Player CallBot calls with amount 40. Remaining stack: 910.
[INFO] maverick: FLOP | Current pot: 200 | Current bet: 40
[INFO] maverick: FLOP | Betting round complete

[INFO] maverick: TURN | Dealt turn. Community cards: ['3♦', '10♠', '8♣', '3♠']
[INFO] maverick: TURN | Player AggroBot bets amount 40. Remaining stack: 870.
[INFO] maverick: TURN | Current pot: 240 | Current bet: 40
[INFO] maverick: TURN | Player CallBot calls with amount 40. Re

...

[INFO] maverick: PRE_FLOP | Player deepseek-r1:8b folds.
[INFO] maverick: PRE_FLOP | Current pot: 90 | Current bet: 40
[INFO] maverick: PRE_FLOP | Player FoldBot folds.
[INFO] maverick: PRE_FLOP | Current pot: 90 | Current bet: 40
[INFO] maverick: PRE_FLOP | Player CallBot calls with amount 20. Remaining stack: 630.
[INFO] maverick: PRE_FLOP | Current pot: 110 | Current bet: 40
[INFO] maverick: PRE_FLOP | Betting round complete

[INFO] maverick: FLOP | Dealt flop. Community cards: ['A♦', 'Q♦', '9♥']
[INFO] maverick: FLOP | Player CallBot checks.
[INFO] maverick: FLOP | Current pot: 110 | Current bet: 0
[INFO] maverick: FLOP | Player AggroBot bets amount 40. Remaining stack: 2020.
[INFO] maverick: FLOP | Current pot: 150 | Current bet: 40
[INFO] maverick: FLOP | Player CallBot calls with amount 40. Remaining stack: 590.
[INFO] maverick: FLOP | Current pot: 190 | Current bet: 40
[INFO] maverick: FLOP | Betting round complete

[INFO] maverick: TURN | Dealt turn. Community cards: ['A♦', 'Q


deepseek-r1:8b is requested to take action.
     Current street: PRE_FLOP
     Button position: 2
     Pot: 70
     Current bet: 40
     Small blind: 10
     Big blind: 20
     Community cards: []
     Player stack: 490
     Player current bet: 0
     Amount to call: 40
     Private cards: ['10♠', '4♣']
     Valid actions: ['FOLD', 'CALL', 'RAISE', 'ALL_IN']
     Min raise BY: 60
     Best hand type: HIGH_CARD
     Best hand score: 100.1004
     Equity (MC): 0.218
     LLMPlayer raw action output: player_id='p3' action_type=<ActionType.FOLD: 1> amount=None
     Player decides to FOLD.



[INFO] maverick: PRE_FLOP | Player deepseek-r1:8b folds.
[INFO] maverick: PRE_FLOP | Current pot: 70 | Current bet: 40
[INFO] maverick: PRE_FLOP | Player FoldBot folds.
[INFO] maverick: PRE_FLOP | Current pot: 70 | Current bet: 40
[INFO] maverick: PRE_FLOP | Player CallBot calls with amount 20. Remaining stack: 820.
[INFO] maverick: PRE_FLOP | Current pot: 90 | Current bet: 40
[INFO] maverick: PRE_FLOP | Betting round complete

[INFO] maverick: FLOP | Dealt flop. Community cards: ['3♠', '9♥', '2♠']
[INFO] maverick: FLOP | Player CallBot checks.
[INFO] maverick: FLOP | Current pot: 90 | Current bet: 0
[INFO] maverick: FLOP | Player AggroBot bets amount 40. Remaining stack: 1860.
[INFO] maverick: FLOP | Current pot: 130 | Current bet: 40
[INFO] maverick: FLOP | Player CallBot calls with amount 40. Remaining stack: 780.
[INFO] maverick: FLOP | Current pot: 170 | Current bet: 40
[INFO] maverick: FLOP | Betting round complete

[INFO] maverick: TURN | Dealt turn. Community cards: ['3♠', '9♥'


deepseek-r1:8b is requested to take action.
     Current street: PRE_FLOP
     Button position: 3
     Pot: 30
     Current bet: 20
     Small blind: 10
     Big blind: 20
     Community cards: []
     Player stack: 490
     Player current bet: 0
     Amount to call: 20
     Private cards: ['4♦', '4♥']
     Valid actions: ['FOLD', 'CALL', 'RAISE', 'ALL_IN']
     Min raise BY: 40
     Best hand type: PAIR
     Best hand score: 204
     Equity (MC): 0.265
     LLMPlayer raw action output: player_id='p3' action_type=<ActionType.FOLD: 1> amount=None
     Player decides to FOLD.



[INFO] maverick: PRE_FLOP | Player deepseek-r1:8b folds.
[INFO] maverick: PRE_FLOP | Current pot: 30 | Current bet: 20
[INFO] maverick: PRE_FLOP | Player FoldBot folds.
[INFO] maverick: PRE_FLOP | Current pot: 30 | Current bet: 20
[INFO] maverick: PRE_FLOP | Player CallBot calls with amount 10. Remaining stack: 1010.
[INFO] maverick: PRE_FLOP | Current pot: 40 | Current bet: 20
[INFO] maverick: PRE_FLOP | Player AggroBot raises to amount 60. Remaining stack: 1720.
[INFO] maverick: PRE_FLOP | Current pot: 80 | Current bet: 60
[INFO] maverick: PRE_FLOP | Player CallBot calls with amount 40. Remaining stack: 970.
[INFO] maverick: PRE_FLOP | Current pot: 120 | Current bet: 60
[INFO] maverick: PRE_FLOP | Betting round complete

[INFO] maverick: FLOP | Dealt flop. Community cards: ['10♠', '8♠', '4♠']
[INFO] maverick: FLOP | Player CallBot checks.
[INFO] maverick: FLOP | Current pot: 120 | Current bet: 0
[INFO] maverick: FLOP | Player AggroBot bets amount 40. Remaining stack: 1680.
[INFO] mav

In [9]:
for player in players:
    print(f"{player.name} - Stack: {player.state.stack}")

CallBot - Stack: 1450
AggroBot - Stack: 0
gpt-5-nano - Stack: 1890
FoldBot - Stack: 660


In [12]:
players[2]._total_call_cost()

0.14052130000000002

In [ ]:
token_usage = players[2]._usage

input_tokens = token_usage["input_tokens"]
output_tokens = token_usage["output_tokens"]
cached_input_tokens = token_usage["input_tokens_details"]["cached_tokens"]

for m in ["gpt-5.2", "gpt-5.1", "gpt-5", "gpt-5-mini", "gpt-5-nano"]:
    cost = openai_call_cost(
        input_tokens=input_tokens,
        output_tokens=output_tokens,
        model=m,
        cached_input_tokens=cached_input_tokens,
    )
    print(f"Total cost on {m}: ${cost:.6f}")